In [4]:
library("renv")
use(
  "Cairo", "dplyr", "tidyr", "duckdb", "dbplyr", "ggplot2",
  "rnaturalearth", "rnaturalearthdata", "sf",
  attach = TRUE, sandbox = FALSE, library = "./renv"
)
custom_palette <- c(
  "#0072B2",  "#D55E00", "#CC79A7", "#009E73", "#E69F00", "#56B4E9",
  "#F0E442", "#999999", "#000000"
)
options(
  repr.plot.width = 6, repr.plot.height = 3.5, repr.plot.res = 300,
  repr.plot.antialias = "subpixel",
  contrasts = c("contr.sum", "contr.poly"),
  ggplot2.discrete.fill = custom_palette,
  ggplot2.discrete.colour = custom_palette
)
theme_set(
  theme_bw() +
    theme(
      text = element_text(family = 'Open Sans'),
      plot.title = element_text(face = "bold", hjust = 0.5),
      legend.position = "top",
      legend.direction = "horizontal",
      legend.box = "vertical",
      panel.grid.major.x = element_blank(),
      panel.grid.minor = element_blank(),
      strip.background = element_blank(),
      axis.title = element_text(size = rel(1.1), face = "bold"),
      axis.text = element_text(size = rel(1.1), face = "bold"),
      strip.text = element_text(size = rel(1.1), face = "bold")
    )
)

# Downloading packages -------------------------------------------------------
- Downloading Cairo from CRAN ...               OK [97.1 Kb in 0.43s]
- Downloading dplyr from CRAN ...               OK [1.2 Mb in 0.54s]
- Downloading generics from CRAN ...            OK [168.2 Kb in 0.15s]
- Downloading tibble from CRAN ...              OK [552.7 Kb in 0.31s]
- Downloading pkgconfig from CRAN ...           OK [5.9 Kb]
- Downloading tidyselect from CRAN ...          OK [99.1 Kb in 0.15s]
- Downloading withr from CRAN ...               OK [105.2 Kb in 0.12s]
- Downloading tidyr from CRAN ...               OK [790.1 Kb in 0.59s]
- Downloading duckdb from CRAN ...              OK [3.6 Mb in 1.6s]
- Downloading DBI from CRAN ...                 OK [1.1 Mb in 0.47s]
- Downloading dbplyr from CRAN ...              OK [743.1 Kb in 0.32s]
- Downloading blob from CRAN ...                OK [10.4 Kb]
- Downloading ggplot2 from CRAN ...             OK [3 Mb in 1.2s]
- Downloading gtable from CRAN ..

ERROR: Error: Error installing package 'curl':
================================

* installing *source* package ‘curl’ ...
** package ‘curl’ successfully unpacked and MD5 sums checked
** using staged installation
Package libcurl was not found in the pkg-config search path.
Perhaps you should add the directory containing `libcurl.pc'
to the PKG_CONFIG_PATH environment variable
No package 'libcurl' found
Package libcurl was not found in the pkg-config search path.
Perhaps you should add the directory containing `libcurl.pc'
to the PKG_CONFIG_PATH environment variable
No package 'libcurl' found
Using PKG_CFLAGS=
Using PKG_LIBS=-lcurl
--------------------------- [ANTICONF] --------------------------------
Configuration failed because libcurl was not found. Try installing:
 * deb: libcurl4-openssl-dev (Debian, Ubuntu, etc)
 * rpm: libcurl-devel (Fedora, CentOS, RHEL)
If libcurl is already installed, check that 'pkg-config' is in your
PATH and PKG_CONFIG_PATH contains a libcurl.pc file. If pk

In [ ]:
con <- dbConnect(
  duckdb(), dbdir = "../processing/globalamfungi_lsu.duckdb", read_only = FALSE
)
inner_join(
  tbl(con, "occurances") |> select(sample_id, taxon_id, abundance, total),
  tbl(con, "samples") |> select(id, longitude, latitude),
  by = join_by(sample_id == id)
) |>
  inner_join(
    tbl(con, "taxa") |> select(accession, Species),
    by = join_by(taxon_id == accession)
  ) |>
  compute(name = "locations")

In [ ]:
count(tbl(con, "locations"), Species) |> collect()
world <- ne_countries(scale = "medium", returnclass = "sf")
p <- ggplot(
  data = tbl(con, "locations")
) +
  geom_sf(
    data = world, fill = "grey", show.legend = FALSE
  ) +
  geom_point(
    aes(x = longitude, y = latitude),
    shape = 21, color = "black", fill = NA, alpha = 0.01
  ) +
  coord_sf(crs = "WGS84 +proj=eqearth", default_crs = "EPSG:4326") +
  theme(
    panel.grid = element_line(color = "lightgray", size = 0.25),
  ) +
  scale_x_continuous(breaks = seq(-180, 180, 10)) +
  scale_y_continuous(breaks = seq(-90, 90, 10)) +
  labs(
    x = element_blank(), y = element_blank()
  )
suppressWarnings(print(p))

In [ ]:
dbDisconnect(con, shutdown = TRUE)